In [1]:
import subprocess
def get_html_from_url(url):
    cmds = ["/Applications/Google Chrome.app/Contents/MacOS/Google Chrome", "--headless", "--dump-dom", "--enable-features=ConversionMeasurement,AttributionReportingCrossAppWeb"]
    cmds.append(url)
    result = subprocess.run(cmds, stdout=subprocess.PIPE)
    text = result.stdout.decode('utf-8')
    return text
    
# t = get_html_from_url("https://www.khanacademy.org/math/algebra/x2f8bb11595b61c86:systems-of-equations/x2f8bb11595b61c86:introduction-to-systems-of-equations/v/trolls-tolls-and-systems-of-equation")
# t
# # result = subprocess.run(['pwd'], stdout=subprocess.PIPE)
# # text = result.stdout.decode('utf-8')
# # text


In [20]:
# unit_url = "https://www.khanacademy.org/math/algebra/x2f8bb11595b61c86:systems-of-equations/x2f8bb11595b61c86:introduction-to-systems-of-equations"
unit_url = "https://www.khanacademy.org/math/algebra/x2f8bb11595b61c86:quadratic-functions-equations"
site = "https://www.khanacademy.org"

In [21]:
from xml.dom.minidom import parseString

html = get_html_from_url(unit_url)



In [22]:
from html.parser import HTMLParser
class Parser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.href = ''
        self.urls = []

    def handle_starttag(self, tag, attrs):
        if (tag == 'a'):
            for (k,v) in attrs:
                if (k == 'href'):
                    self.href = v
        elif (tag == 'span'):
            for (k,v) in attrs:
                if (k == 'aria-label' and v == 'Video'):
                    self.urls.append(self.href)            



parser = Parser()
parser.feed(html)
parser.urls


['/math/algebra/x2f8bb11595b61c86:quadratic-functions-equations/x2f8bb11595b61c86:intro-parabolas/v/parabolas-intro',
 '/math/algebra/x2f8bb11595b61c86:quadratic-functions-equations/x2f8bb11595b61c86:intro-parabolas/v/interpret-parabola-context',
 '/math/algebra/x2f8bb11595b61c86:quadratic-functions-equations/x2f8bb11595b61c86:intro-parabolas/v/interpret-a-quadratic-graph',
 '/math/algebra/x2f8bb11595b61c86:quadratic-functions-equations/x2f8bb11595b61c86:factored-form-quadratics/v/zero-product-property',
 '/math/algebra/x2f8bb11595b61c86:quadratic-functions-equations/x2f8bb11595b61c86:factored-form-quadratics/v/graphing-quadratics-in-factored-form',
 '/math/algebra/x2f8bb11595b61c86:quadratic-functions-equations/x2f8bb11595b61c86:factored-form-quadratics/v/quadratic-word-problems-factored-form',
 '/math/algebra/x2f8bb11595b61c86:quadratic-functions-equations/x2f8bb11595b61c86:untitled-1082/v/simple-quadratic-equation',
 '/math/algebra/x2f8bb11595b61c86:quadratic-functions-equations/x2f

In [32]:
video_url = site + parser.urls[0]
video_html = get_html_from_url(video_url)




[0521/115618.386214:WARNING:runtime_features.cc(730)] AttributionReportingCrossAppWeb cannot be enabled in this configuration. Use --enable-features=ConversionMeasurement,AttributionReportingCrossAppWeb in addition.
[0521/115618.527860:WARNING:runtime_features.cc(730)] AttributionReportingCrossAppWeb cannot be enabled in this configuration. Use --enable-features=ConversionMeasurement,AttributionReportingCrossAppWeb in addition.


In [49]:
class VideoParser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.start_h1 = False
        self.start_div = False
        self.start_iframe = False
        
        self.name = ''
        self.desc = ''

    def handle_starttag(self, tag, attrs):
        if (tag == 'h1'):
            self.start_h1 = True       
        elif (tag == 'div'):
            for (k,v) in attrs:
                if (k == 'id' and v == 'ka-videoPageTabs-tabbedpanel-content'):
                    self.start_div = True
        elif (tag == 'iframe'):
            print(attrs)
            for (k,v) in attrs:
                if(k == 'data-youtubeid'):
                    print(v)

    def handle_endtag(self, tag):
        if (tag == 'h1'):
            self.start_h1 = False
        elif (tag == 'div'):
            self.start_div = False
            
    def handle_data(self, data):
        if self.start_h1:
            self.name = data.title()
        if self.start_div:
            self.desc += data

video_parser = VideoParser()
video_parser.feed(video_html)
print(video_parser.name, video_parser.desc)




[('src', 'https://www.googletagmanager.com/ns.html?id=GTM-WSX5PBP'), ('height', '0'), ('width', '0'), ('style', 'display:none;visibility:hidden')]
Systems Of Equations: Trolls, Tolls (1 Of 2) A troll forces us to use algebra to figure out the make-up of his currency. We end up setting up a system of equations. Created by Sal Khan.


In [5]:
def get_searched_value(video_html, searchStr):
    x = video_html.find(searchStr)
    start = video_html.find('"', x+len(searchStr))
    end = video_html.find('"', start+1)
    return video_html[start+1:end]


def get_topic(video_html):
    desc =  get_searched_value(video_html, '"translatedDescription":')
    name = get_searched_value(video_html, '"translatedTitle":').title()
    videoID = get_searched_value(video_html, '"translatedYoutubeId":')
    return {
        "name": name,
        "description": desc,
        "texts": ["text1.md"],
        "video_parts": [{
            "id": videoID
        }],
        "node_type": "lecture",
        "parent_id": "khan-algebra-1",
        "thumbnail": ""
    }

In [23]:
import json
topics = []
for url in parser.urls:
    video_url = site + url
    video_html = get_html_from_url(video_url)
    topic = get_topic(video_html)
    print(json.dumps(topic, indent=4))
    topics.append(topic)
    


    
    
    
    

{
    "name": "<Html><Head></Head><Body></Body></Html>",
    "description": "<html><head></head><body></body></html>",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "<html><head></head><body></body></html>"
        }
    ],
    "node_type": "lecture",
    "parent_id": "khan-algebra-1",
    "thumbnail": ""
}


[0529/193636.153787:ERROR:gles2_command_buffer_stub.cc(300)] ContextResult::kFatalFailure: Failed to create context.


{
    "name": "Interpreting A Parabola In Context",
    "description": "Given a parabola that models a context, we can relate key features of the parabola \u2014\u00a0like the y-intercept, vertex, and x-intercepts \u2014\u00a0to what they represent in the given context.",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "_Bk6XkV9O_0"
        }
    ],
    "node_type": "lecture",
    "parent_id": "khan-algebra-1",
    "thumbnail": ""
}
{
    "name": "Interpret A Quadratic Graph",
    "description": "We can interpret what the features of a graph of a quadratic model mean in terms of a given context.",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "Tqxu53deWCo"
        }
    ],
    "node_type": "lecture",
    "parent_id": "khan-algebra-1",
    "thumbnail": ""
}
{
    "name": "Zero Product Property",
    "description": "The zero product property states that if a\u22c5b=0 then either a or b equal zero. This 

[0529/195205.324412:WARNING:spdy_session.cc(2979)] Received HEADERS for invalid stream 19


{
    "name": "Quadratic Equations Word Problem: Box Dimensions",
    "description": "Sal solves a volume problem using a quadratic equation.",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "vl9o9XEfXtw"
        }
    ],
    "node_type": "lecture",
    "parent_id": "khan-algebra-1",
    "thumbnail": ""
}
{
    "name": "Text/Javascript",
    "description": "text/javascript",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "text/javascript"
        }
    ],
    "node_type": "lecture",
    "parent_id": "khan-algebra-1",
    "thumbnail": ""
}
{
    "name": "Worked Example: Quadratic Formula (Example 2)",
    "description": "Sal solves the equation -x^2+8x=1 by first bringing it to standard form and then using the quadratic formula.",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "CLrImGKeuEI"
        }
    ],
    "node_type": "lecture",
    "parent_id": "khan-al

...

[0530/031510.190510:WARNING:dns_config_service_posix.cc(196)] Failed to read DnsConfig.
[0530/031510.361966:WARNING:dns_config_service_posix.cc(196)] Failed to read DnsConfig.


{
    "name": "<Html><Head></Head><Body></Body></Html>",
    "description": "<html><head></head><body></body></html>",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "<html><head></head><body></body></html>"
        }
    ],
    "node_type": "lecture",
    "parent_id": "khan-algebra-1",
    "thumbnail": ""
}


[0530/031510.429953:WARNING:dns_config_service_posix.cc(196)] Failed to read DnsConfig.
[0530/031510.602931:WARNING:dns_config_service_posix.cc(196)] Failed to read DnsConfig.


{
    "name": "<Html><Head></Head><Body></Body></Html>",
    "description": "<html><head></head><body></body></html>",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "<html><head></head><body></body></html>"
        }
    ],
    "node_type": "lecture",
    "parent_id": "khan-algebra-1",
    "thumbnail": ""
}
{
    "name": "<Html><Head></Head><Body></Body></Html>",
    "description": "<html><head></head><body></body></html>",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "<html><head></head><body></body></html>"
        }
    ],
    "node_type": "lecture",
    "parent_id": "khan-algebra-1",
    "thumbnail": ""
}


[0530/031510.662920:WARNING:dns_config_service_posix.cc(196)] Failed to read DnsConfig.
